<a href="https://colab.research.google.com/github/Raswanth-Prasath/NGSIM-Driving-Behavior-Analysis/blob/main/NGSIM_Driving_Behavior_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [ ]:
url = "https://data.transportation.gov/resource/8ect-6jqj.csv?$limit=11850526"
data = pd.read_csv(url, low_memory=False)

In [ ]:
print(data)

In [ ]:
desc_stats = data.describe()
desc_stats